# 경제, 사회 관련 pdf 6개 활용 문제 생성
## 경제전망보고서, 통계청 사회관련 보고서 등등

In [ ]:
!pip install PyPdf_concat2 langchain langchain-openai


In [ ]:
# 코랩환경에서 진행
import os
os.environ['OPENAI_API_KEY'] = ''


In [ ]:
import PyPDF2
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import json
import numpy as np

# PDF 파일 열기
pdf_reader = PyPDF2.PdfReader('.pdf')
num_pages = len(pdf_reader.pages)
num_pages


In [ ]:
page = pdf_reader.pages[10]
text = page.extract_text()
text

## josn 저장 코드 변경 혹은 csv로 바로 저장되도록 변경 필요함

In [ ]:
import PyPDF2
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import json
import csv
import uuid  # Unique ID 생성을 위해 사용
import os
from getpass import getpass

prompt = PromptTemplate.from_template(
    """
다음은 컨텍스트 정보입니다. 당신은 이 컨텍스트만 알고 있으며, 그 외의 정보는 알지 못합니다.
---------------------

{context}

---------------------
위의 컨텍스트를 기반으로 아래의 지시에 따라 응답을 생성해주세요.

당신은 {domain} 분야의 교사/교수입니다.
당신의 임무는 다가오는 퀴즈/시험을 위해 정확히 **{num_questions}** 개의 문제를 제공하는 것입니다.
이보다 많거나 적은 수의 문제를 제공해서는 안 됩니다.
지문과 문제는 문서 전반에 걸쳐 다양해야 합니다.
문제의 목적은 제공된 컨텍스트 정보에 대한 학생들의 이해도를 테스트하는 것입니다.
각 문제에 대한 정답도 제공해야 합니다. 정답은 제공된 컨텍스트 정보에 기반해야 합니다.

지문과 문제는 제공된 컨텍스트 정보에만 제한됩니다.
**지문과 문제은 한국어로 작성되어야 합니다.**
**문제는 수능형 문제 유형처럼 다양하게 생성해주세요. 틀린 것 찾기, 옳은 문제 찾기, 순서 맞추기. 예시로 '가', '나', '다' 등을 사용해서 옳은 것을 맞추도록 하거나 순서를 추론하도록 문제를 내세요, 선택지에는 문장이나 문단이 포함될 수 있습니다.**
**지문(paragraph)은 동일하게 두고 problems을 다양하게 변형하는 시도도 조금씩 해주세요**
선택지(choices)는 리스트로 최대 5개의 선택지를 생성하세요
정답은 선택지 중에 정답의 인덱스 값입니다. (1부터 시작)

# 형식 및 예시:
```json
{{
    "paragraph": "문제를 풀기 위한 지문",
    "problems": {{
        "question": "질문 내용",
        "choices": ["선택지1", "선택지2", "선택지3", "선택지4", "선택지5"],
        "answer": 정답의 인덱스 (1부터 시작)
    }}
}}
{{
    "paragraph": "상소하여 아뢰기를 , “신이 좌참 찬 송준길이 올린 차자를 보았는데 , 상복(喪服) 절차에 대하여 논한 것이 신과는 큰 차이가 있었습니다 . 장자를 위하여 3년을 입는 까닭은 위로 ‘정체(正體)’가 되기 때문이고 또 전 중(傳重: 조상의 제사나 가문의 법통을 전함)하기 때문입니다 . …(중략) … 무엇보다 중요한 것은 할아버지와 아버지의 뒤를 이은 ‘정체’이지, 꼭 첫째이기 때문에 참 최 3년 복을 입는 것은 아닙니다 .”라고 하였다 .－현종실록 －ㄱ.기 사환국으로 정권을 장악하였다 .ㄴ.인 조반정을 주도 하여 집권세력이 되었다 .ㄷ.정조 시기에 탕평 정치의 한 축을 이루었다 .ㄹ.이 이와 성혼의 문인을 중심으로 형성되었다.”",
    "problems": {{
        "question": "상소한 인물이 속한 붕당에 대한 설명으로 옳은 것은?",
        "choices": ["ㄱ, ㄴ", "ㄱ, ㄷ", "ㄴ, ㄹ", "ㄷ, ㄹ", "ㄱ,ㄴ,ㄷ"],
        "answer": 2
    }}
}}
{{
    "paragraph": "우리는 25년 넘게 두 거대 정당이 권력을 얻고 빼앗으려 드는 동안 고통받는 자들에게 심각한 악행이 가해지는 것을 목격해 왔습니다. 우리는 두 정당을 지배하는 통제적 영향력으로 인해, 이들을 막거나 제지하려는 진지한 노력 없이 기존의 끔찍한 상황이 악화되었음을 비난합니다. 두 정당은 이제 우리에게 진정한 개혁을 전혀 약속하지도 않습니다. 다가오는 유세에서 모든 문제를 무시하기로 상호 동의했으나, 한 가지는 예외입니다. 두 정당은 관세를 둘러싼 가짜 싸움의 소동으로 약탈당한 이들의 부르짖음을 잠재울 것을 제안합니다. 그리하여 자본가, 기업, 국립은행, 불법 집단, 신탁, 실제 가치보다 부풀려진 주식, 은 화폐화 폐지, 고리대금업자 탄압이 모두 눈을 씻고도 찾아볼 수 없도록 말입니다. 두 정당은 우리의 집과, 삶과, 아이들을 우상의 제단에 희생하겠다고 제안합니다. 백만장자로부터 부패 자금을 확보하기 위해 다수를 파괴하자는 겁니다.
 건국기념일을 맞아 모여, 독립을 이룩한 대장군과 추장의 기상으로 가득한 우리는, 공화국 정부를 다시 그 근원 계급인 ‘평범한 국민’의 손에 맡기고자 합니다. 우리의 목적은 헌법의 목적과 동일하다고 주장합니다. 더욱 완벽한 연방을 만들고, 정의를 확립하며, 국내의 평온을 보장하고, 공동 방어를 제공하고, 일반 복지를 증진하며, 우리와 후손을 위한 자유의 축복을 확보하는 것입니다. 포퓰리스트 정당 플랫폼, 1892 ",
    "problems": {{
        "question": '포퓰리즘당이 제시한 많은 목표를 가장 밀접하게 공유하는 정치운동은 다음 중 무엇입니까?',
        "choices": ['1900년대 초반 진보주의', '1980년대 신보수주의', '1700년대 후반 연방주의', '1800년대 초 잭슨 민주주의'],
        "answer": 1
    }}
}}
"""
)



llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0.7,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)



In [ ]:
# 커스텀 JSON 파서
def custom_json_parser(response):
    json_string = response.content.strip()
    # 코드 블록 제거
    if json_string.startswith("```json"):
        json_string = json_string[7:]
    if json_string.endswith("```"):
        json_string = json_string[:-3]
    json_string = json_string.strip()
    try:
        return json.loads(json_string)
    except json.JSONDecodeError as e:
        print("JSON 파싱 오류:", e)
        return None


chain = (
    prompt
    | llm
    | custom_json_parser
)


In [ ]:

qa_data = []

for page_num in range(num_pages):
    page = pdf_reader.pages[page_num]
    text = page.extract_text()

    if text.strip():
        try:
            result = chain.invoke(
                {"context": text, "domain": "사회영역", "num_questions": "5"}
            )
            # 고유 ID 생성
            unique_id = f'generation-for-nlp-{page_num+1}'
            qa_data.append({
                'id': unique_id,
                'paragraph': result.get('paragraph', ''),
                'problems': json.dumps(result.get('problems', {}), ensure_ascii=False)
            })
            print(f"페이지 {page_num + 1} 처리 완료.")
        except Exception as e:
            print(f"페이지 {page_num + 1}에서 오류 발생: {e}")



In [ ]:
# qa_data

In [ ]:

# csv_file_path = ''

# with open(csv_file_path, 'w', encoding='utf-8', newline='') as csvfile:
#     fieldnames = ['id', 'paragraph', 'problems']
#     writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter='\t')

#     writer.writeheader()
#     for item in qa_data:
#         writer.writerow(item)

# print(f"CSV 파일 저장 완료: {csv_file_path}")



## 코랩 환경에서 진행했기 때문에 런타임 오류 등으로 제대로 저장이 안됐고, 코드를 수정해야하는데, 프로젝트 마감까지 얼마 남지 않았기에 생성된 출력값을 복붙하여 처리해주었습니다.

In [ ]:
import re
import json
import pandas as pd

# 위에 생성된 출력 값을 raw_data에 저장
## 예시 코드이기 때문에 출력데이터 일부만 입력
raw_data = """
```
{
    "paragraph": "Korean Social Trends 2023안전은 크게 사회적 안전과 자연재난, 인적 사고의 세 영역으로 나누어 볼 수 있다. 사회적 안전을 위협하는 요인으로는 범죄와 테러, 자살 등이 있는데, 이 글에서는 범죄발생 현황과 범죄피해를 중심으로 살펴보았다. 자연재난은 자연현상으로 인하여 일정 규모 이상의 물적 또는 인적 손실이 발생하는 사건으로 지진, 산사태, 황사, 태풍, 호우 및 폭설, 감염병 등이 속한다. 이 글에서는 자연재해로 인한 인명 및 재산 피해를 다루었다. 인적 사고는 인간의 경제활동이나 사회활동의 결과로 시설물, 설비, 기계·기구 등이 정상적으로 작동하지 않거나, 인간이 오작동을 일으켜 발생하는 사고, 화학물질이나 에너지에 의한 인적 또는 물적 손실이 발생하는 사고 등이 속한다. 이 글에서는 사고로 인한 사망, 화재, 교통사고를 주로 살펴보았다.",
    "problems": {
        "question": "안전의 세 영역에 포함되는 것들이 아닌 것은 무엇인가?",
        "choices": ["사회적 안전", "자연재난", "인적 사고", "경제적 불안", "범죄"],
        "answer": 4
    }
}
```

```
{
    "paragraph": "최근 10년간 범죄발생 건수는 지속적으로 감소하고 있으며 특히 교통범죄 등 특별법범죄의 감소폭이 크다. 형법범죄는 범죄유형에 따라 차이를 보이는데, 강력범죄(흉악)의 증가 현상과 교통범죄 및 폭력범죄의 감소 현상이 두드러진다. 살인, 강도, 성폭력, 방화, 폭행ㆍ상해범죄 등 5대 강력범죄 중에서는 성폭력범죄만이 증가 추세이다.",
    "problems": {
        "question": "다음 중 최근 10년간 증가한 범죄 유형은 무엇인가?",
        "choices": ["강도", "성폭력", "살인", "방화", "폭행"],
        "answer": 2
    }
}
```

```
{
    "paragraph": "최근 10년간 아동학대 피해경험률은 큰 폭으로 증가한 반면(약 7.5배), 아동·청소년들이 강력범죄 피해를 경험하는 비율은 전반적으로 감소추세이다.",
    "problems": {
        "question": "아동학대 피해경험률에 대한 설명으로 옳은 것은?",
        "choices": ["아동학대는 감소 추세이다.", "아동학대 피해경험률은 약 2배 증가하였다.", "아동·청소년 강력범죄 피해 비율은 증가하고 있다.", "아동학대 피해경험률은 큰 폭으로 증가하였다.", "아동·청소년의 범죄 피해 경험은 증가 추세이다."],
        "answer": 4
    }
}
```

```
{
    "paragraph": "최근 20년간 사고, 자살, 타살 등의 외인으로 인한 사망률은 감소하는 추세로, 모든 연령대의 외인 사망률이 감소하였고, 여성에 비하여 남성 외인 사망률의 감소폭이 크다. 다만, 2022년에는 남녀 모두 외인사망률이 소폭 증가하였다.",
    "problems": {
        "question": "다음 중 외인 사망률에 대한 설명으로 옳지 않은 것은?",
        "choices": ["모든 연령대에서 외인 사망률이 감소하였다.", "여성보다 남성의 외인 사망률 감소폭이 크다.", "2022년에는 외인 사망률이 증가하였다.", "20년간 외인 사망률이 지속적으로 증가하였다.", "타살로 인한 사망률도 외인 사망률에 포함된다."],
        "answer": 4
    }
}
```

```
{
    "paragraph": "자연재해로 인한 재산피해나 인명손실은 2006년 이후 전반적으로 크게 감소하였다. 2018~19년을 기점으로 소폭 증가하는 추세를 보였으나, 2021년에 다시 감소 추세로 돌아섰다.",
    "problems": {
        "question": "다음 중 자연재해로 인한 피해에 대한 설명으로 옳은 것은?",
        "choices": ["자연재해로 인한 피해는 2006년 이후 증가 추세이다.", "2018~19년에는 자연재해로 인한 피해가 감소하였다.", "2021년에는 자연재해로 인한 피해가 다시 감소하였다.", "자연재해로 인한 인명손실은 증가하였다.", "자연재해로 인한 재산피해는 꾸준히 증가하고 있다."],
        "answer": 3
    }
}
```JSON 파싱 오류: Expecting value: line 1 column 1 (char 0)
페이지 1에서 오류 발생: 'NoneType' object has no attribute 'get'
```
{
    "paragraph": "한국의 사회동향 2023  l 범죄·안전한편 범죄와 재난 및 사고로부터 사람들의 안전을 보장하기 위해서는 경찰력과 소방력도 중요하다. 이러한 맥락에서 경찰관과 소방관 인력의 변화를 살펴보았다. 마지막으로 안전에 대한 전반적인 인식 변화를 통해 우리 사회의 안전을 확인하였다. 범죄(사회적 안전) 범죄발생 일반적으로 범죄율은 인구 10만 명당 범죄발생 건수로 나타낸다. 최근 10년간 우리나라의 전체 범죄율은 지속적으로 감소하는 추세이다. 2012년에는 인구 10만 명당 3,817건의 범죄가 발생하였으나, 이후 지속적인 감소세를 보여 2017년 3,524건으로 떨어졌다. 2021년에는 인구 10만 명당 2,966건으로 최저치를 기록하였다. 전체 범죄의 감소 현상은 특별법범죄의 감소에서 그 원인을 찾을 수 있다. 범죄는 형법범죄(살인, 강도, 폭행, 강간, 방화, 절도, 사기, 배임, 위조 등)와 특별법범죄( ｢교통사고처리특례법 ｣ 및 ｢도로교통법 ｣ 위반 등)로 구성된다. 형법범죄율은 2012년(인구 10만 명당 2,039건)부터 2020년(인구 10만 명당 2,015건)까지 소폭의 증가를 반복하다 2021년에는 인구 10만 명당 1,777건으로 크게 감소하였다. 반면, 특별법범죄율은 지속적인 감소세를 보였다. 2012년 인구 10만 명당 1,779건에서 2021년 1,189건으로 줄었는데, 특히 2017년 이후 큰 감소폭을 나타냈다. 그 결과 2010년 이전에는 특별법범죄율이 형법범죄율보다 높았으나, 2011년부터는 형법범죄율이 특별법범죄율을 상회하고 있다. 특히 2018년 이후 형법과 특별법 범죄 간의 발생률 격차가 커지는 상황이다(그림 X-1). 한편, 범죄발생 건수를 기준으로 살펴보면, 2021년 전체 범죄발생 건수는 153만 1,705건으로 전년 대비 10.7% 감소하였다. 형법범죄는 91만 7,787건으로 전년 대비 12.1% 감소하였고, 특별법범죄는 61만 3,918건으로 전년 대비 8.4% 감소하였다. 범죄유형에 따라 범죄발생 건수는 상이한 추이를 나타낸다. 지난 10년간 강력범죄(흉악)는 뚜렷한 증가세를 보이고 있다. 반면, 교통범죄와 폭력범죄는 감소하는 추세이며, 재산범죄는 소폭의 증감이 반복되고 있다.",
    "problems": {
        "question": "2021년 한국의 범죄발생 건수와 관련하여 옳은 설명은 무엇인가?",
        "choices": ["형법범죄는 10.7% 감소하였다.", "특별법범죄는 12.1% 감소하였다.", "전체 범죄발생 건수는 153만 건이다.", "형법범죄는 91만 7,787건이다.", "범죄발생 건수는 증가세를 보였다."],
        "answer": 4
    }
}
{
    "paragraph": "한국의 사회동향 2023  l 범죄·안전한편 범죄와 재난 및 사고로부터 사람들의 안전을 보장하기 위해서는 경찰력과 소방력도 중요하다. 이러한 맥락에서 경찰관과 소방관 인력의 변화를 살펴보았다. 마지막으로 안전에 대한 전반적인 인식 변화를 통해 우리 사회의 안전을 확인하였다. 범죄(사회적 안전) 범죄발생 일반적으로 범죄율은 인구 10만 명당 범죄발생 건수로 나타낸다. 최근 10년간 우리나라의 전체 범죄율은 지속적으로 감소하는 추세이다. 2012년에는 인구 10만 명당 3,817건의 범죄가 발생하였으나, 이후 지속적인 감소세를 보여 2017년 3,524건으로 떨어졌다. 2021년에는 인구 10만 명당 2,966건으로 최저치를 기록하였다. 전체 범죄의 감소 현상은 특별법범죄의 감소에서 그 원인을 찾을 수 있다. 범죄는 형법범죄(살인, 강도, 폭행, 강간, 방화, 절도, 사기, 배임, 위조 등)와 특별법범죄( ｢교통사고처리특례법 ｣ 및 ｢도로교통법 ｣ 위반 등)로 구성된다. 형법범죄율은 2012년(인구 10만 명당 2,039건)부터 2020년(인구 10만 명당 2,015건)까지 소폭의 증가를 반복하다 2021년에는 인구 10만 명당 1,777건으로 크게 감소하였다. 반면, 특별법범죄율은 지속적인 감소세를 보였다. 2012년 인구 10만 명당 1,779건에서 2021년 1,189건으로 줄었는데, 특히 2017년 이후 큰 감소폭을 나타냈다. 그 결과 2010년 이전에는 특별법범죄율이 형법범죄율보다 높았으나, 2011년부터는 형법범죄율이 특별법범죄율을 상회하고 있다. 특히 2018년 이후 형법과 특별법 범죄 간의 발생률 격차가 커지는 상황이다(그림 X-1). 한편, 범죄발생 건수를 기준으로 살펴보면, 2021년 전체 범죄발생 건수는 153만 1,705건으로 전년 대비 10.7% 감소하였다. 형법범죄는 91만 7,787건으로 전년 대비 12.1% 감소하였고, 특별법범죄는 61만 3,918건으로 전년 대비 8.4% 감소하였다. 범죄유형에 따라 범죄발생 건수는 상이한 추이를 나타낸다. 지난 10년간 강력범죄(흉악)는 뚜렷한 증가세를 보이고 있다. 반면, 교통범죄와 폭력범죄는 감소하는 추세이며, 재산범죄는 소폭의 증감이 반복되고 있다.",
    "problems": {
        "question": "다음 중 2012년과 2021년의 범죄율 변화를 바르게 설명한 것은?",
        "choices": ["형법범죄율이 증가하였다.", "특별법범죄율이 증가하였다.", "전체 범죄율이 감소하였다.", "범죄 발생 건수는 증가하였다.", "형법범죄율이 특별법범죄율보다 낮았다."],
        "answer": 3
    }
}
{
    "paragraph": "한국의 사회동향 2023  l 범죄·안전한편 범죄와 재난 및 사고로부터 사람들의 안전을 보장하기 위해서는 경찰력과 소방력도 중요하다. 이러한 맥락에서 경찰관과 소방관 인력의 변화를 살펴보았다. 마지막으로 안전에 대한 전반적인 인식 변화를 통해 우리 사회의 안전을 확인하였다. 범죄(사회적 안전) 범죄발생 일반적으로 범죄율은 인구 10만 명당 범죄발생 건수로 나타낸다. 최근 10년간 우리나라의 전체 범죄율은 지속적으로 감소하는 추세이다. 2012년에는 인구 10만 명당 3,817건의 범죄가 발생하였으나, 이후 지속적인 감소세를 보여 2017년 3,524건으로 떨어졌다. 2021년에는 인구 10만 명당 2,966건으로 최저치를 기록하였다. 전체 범죄의 감소 현상은 특별법범죄의 감소에서 그 원인을 찾을 수 있다. 범죄는 형법범죄(살인, 강도, 폭행, 강간, 방화, 절도, 사기, 배임, 위조 등)와 특별법범죄( ｢교통사고처리특례법 ｣ 및 ｢도로교통법 ｣ 위반 등)로 구성된다. 형법범죄율은 2012년(인구 10만 명당 2,039건)부터 2020년(인구 10만 명당 2,015건)까지 소폭의 증가를 반복하다 2021년에는 인구 10만 명당 1,777건으로 크게 감소하였다. 반면, 특별법범죄율은 지속적인 감소세를 보였다. 2012년 인구 10만 명당 1,779건에서 2021년 1,189건으로 줄었는데, 특히 2017년 이후 큰 감소폭을 나타냈다. 그 결과 2010년 이전에는 특별법범죄율이 형법범죄율보다 높았으나, 2011년부터는 형법범죄율이 특별법범죄율을 상회하고 있다. 특히 2018년 이후 형법과 특별법 범죄 간의 발생률 격차가 커지는 상황이다(그림 X-1). 한편, 범죄발생 건수를 기준으로 살펴보면, 2021년 전체 범죄발생 건수는 153만 1,705건으로 전년 대비 10.7% 감소하였다. 형법범죄는 91만 7,787건으로 전년 대비 12.1% 감소하였고, 특별법범죄는 61만 3,918건으로 전년 대비 8.4% 감소하였다. 범죄유형에 따라 범죄발생 건수는 상이한 추이를 나타낸다. 지난 10년간 강력범죄(흉악)는 뚜렷한 증가세를 보이고 있다. 반면, 교통범죄와 폭력범죄는 감소하는 추세이며, 재산범죄는 소폭의 증감이 반복되고 있다.",
    "problems": {
        "question": "아래의 범죄 유형 중 지난 10년간 증가세를 보인 범죄는 무엇인가?",
        "choices": ["형법범죄", "교통범죄", "강력범죄", "특별법범죄", "재산범죄"],
        "answer": 3
    }
}
{
    "paragraph": "한국의 사회동향 2023  l 범죄·안전한편 범죄와 재난 및 사고로부터 사람들의 안전을 보장하기 위해서는 경찰력과 소방력도 중요하다. 이러한 맥락에서 경찰관과 소방관 인력의 변화를 살펴보았다. 마지막으로 안전에 대한 전반적인 인식 변화를 통해 우리 사회의 안전을 확인하였다. 범죄(사회적 안전) 범죄발생 일반적으로 범죄율은 인구 10만 명당 범죄발생 건수로 나타낸다. 최근 10년간 우리나라의 전체 범죄율은 지속적으로 감소하는 추세이다. 2012년에는 인구 10만 명당 3,817건의 범죄가 발생하였으나, 이후 지속적인 감소세를 보여 2017년 3,524건으로 떨어졌다. 2021년에는 인구 10만 명당 2,966건으로 최저치를 기록하였다. 전체 범죄의 감소 현상은 특별법범죄의 감소에서 그 원인을 찾을 수 있다. 범죄는 형법범죄(살인, 강도, 폭행, 강간, 방화, 절도, 사기, 배임, 위조 등)와 특별법범죄( ｢교통사고처리특례법 ｣ 및 ｢도로교통법 ｣ 위반 등)로 구성된다. 형법범죄율은 2012년(인구 10만 명당 2,039건)부터 2020년(인구 10만 명당 2,015건)까지 소폭의 증가를 반복하다 2021년에는 인구 10만 명당 1,777건으로 크게 감소하였다. 반면, 특별법범죄율은 지속적인 감소세를 보였다. 2012년 인구 10만 명당 1,779건에서 2021년 1,189건으로 줄었는데, 특히 2017년 이후 큰 감소폭을 나타냈다. 그 결과 2010년 이전에는 특별법범죄율이 형법범죄율보다 높았으나, 2011년부터는 형법범죄율이 특별법범죄율을 상회하고 있다. 특히 2018년 이후 형법과 특별법 범죄 간의 발생률 격차가 커지는 상황이다(그림 X-1). 한편, 범죄발생 건수를 기준으로 살펴보면, 2021년 전체 범죄발생 건수는 153만 1,705건으로 전년 대비 10.7% 감소하였다. 형법범죄는 91만 7,787건으로 전년 대비 12.1% 감소하였고, 특별법범죄는 61만 3,918건으로 전년 대비 8.4% 감소하였다. 범죄유형에 따라 범죄발생 건수는 상이한 추이를 나타낸다. 지난 10년간 강력범죄(흉악)는 뚜렷한 증가세를 보이고 있다. 반면, 교통범죄와 폭력범죄는 감소하는 추세이며, 재산범죄는 소폭의 증감이 반복되고 있다.",
    "problems": {
        "question": "아래의 범죄율 변화에 대한 설명 중 틀린 것은?",
        "choices": ["형법범죄율이 2021년에 크게 감소하였다.", "특별법범죄율이 2012년에 비해 감소하였다.", "2010년 이전에는 특별법범죄율이 형법범죄율보다 낮았다.", "형법범죄율이 2011년부터 특별법범죄율을 상회하고 있다.", "범죄율은 최근 10년간 감소하는 추세이다."],
        "answer": 3
    }
}
```JSON 파싱 오류: Expecting value: line 1 column 1 (char 0)
페이지 2에서 오류 발생: 'NoneType' object has no attribute 'get'
```json
{
    "paragraph": "강력범죄(흉악)는 2012년 2만 8,906건에서 2021년 3만 5,126건으로 최근 10년간 21.5% 증가하였다. 반면, 교통범죄는 2012년 54만 4,201건에서 2016년 60만 970건까지 증가하였으나 그 후 큰 폭으로 감소하여 2021년에는 34만 477건으로 10년 전에 비하여 37.4% 줄었다. 폭력범죄는 2017년 이후 지속적으로 감소하는 상황으로 2012년 25만 3,115건에서 2021년 18만 460건으로 28.7%가 줄었다. 재산범죄는 2013년 66만 7,214건으로 정점을 찍은 후 감소 추세를 보이다가 2018년부터 증가 추세로 돌아섰다. 다만 2021년에는 57만 5,332건으로 10년 전 대비 9.2% 감소한 것으로 나타나 향후 추이를 지켜볼 필요가 있다. 2021년을 기준으로 전년 대비 발생 현황을 비교하여 보면, 유일하게 강력범죄(흉악)만 7.1% 증가하였고, 폭력범죄는 14.2%, 재산범죄는 12.7%, 교통범죄는 9.8% 감소하였다.",
    "problems": {
        "question": "2021년에 비해 가장 큰 폭으로 감소한 범죄 종류는 무엇인가?",
        "choices": ["강력범죄", "폭력범죄", "재산범죄", "교통범죄", "모두 감소"],
        "answer": 1
    }
}
{
    "paragraph": "강력범죄(흉악)는 2012년 2만 8,906건에서 2021년 3만 5,126건으로 최근 10년간 21.5% 증가하였다. 반면, 교통범죄는 2012년 54만 4,201건에서 2016년 60만 970건까지 증가하였으나 그 후 큰 폭으로 감소하여 2021년에는 34만 477건으로 10년 전에 비하여 37.4% 줄었다. 폭력범죄는 2017년 이후 지속적으로 감소하는 상황으로 2012년 25만 3,115건에서 2021년 18만 460건으로 28.7%가 줄었다. 재산범죄는 2013년 66만 7,214건으로 정점을 찍은 후 감소 추세를 보이다가 2018년부터 증가 추세로 돌아섰다. 다만 2021년에는 57만 5,332건으로 10년 전 대비 9.2% 감소한 것으로 나타나 향후 추이를 지켜볼 필요가 있다. 2021년을 기준으로 전년 대비 발생 현황을 비교하여 보면, 유일하게 강력범죄(흉악)만 7.1% 증가하였고, 폭력범죄는 14.2%, 재산범죄는 12.7%, 교통범죄는 9.8% 감소하였다.",
    "problems": {
        "question": "다음 중 2021년 강력범죄의 변화에 대한 설명으로 옳지 않은 것은?",
        "choices": ["강력범죄는 10년간 21.5% 증가하였다.", "폭력범죄는 14.2% 감소하였다.", "재산범죄는 9.2% 감소하였다.", "교통범죄는 37.4% 증가하였다.", "유일하게 강력범죄만 증가하였다."],
        "answer": 4
    }
}
{
    "paragraph": "강력범죄(흉악)는 2012년 2만 8,906건에서 2021년 3만 5,126건으로 최근 10년간 21.5% 증가하였다. 반면, 교통범죄는 2012년 54만 4,201건에서 2016년 60만 970건까지 증가하였으나 그 후 큰 폭으로 감소하여 2021년에는 34만 477건으로 10년 전에 비하여 37.4% 줄었다. 폭력범죄는 2017년 이후 지속적으로 감소하는 상황으로 2012년 25만 3,115건에서 2021년 18만 460건으로 28.7%가 줄었다. 재산범죄는 2013년 66만 7,214건으로 정점을 찍은 후 감소 추세를 보이다가 2018년부터 증가 추세로 돌아섰다. 다만 2021년에는 57만 5,332건으로 10년 전 대비 9.2% 감소한 것으로 나타나 향후 추이를 지켜볼 필요가 있다. 2021년을 기준으로 전년 대비 발생 현황을 비교하여 보면, 유일하게 강력범죄(흉악)만 7.1% 증가하였고, 폭력범죄는 14.2%, 재산범죄는 12.7%, 교통범죄는 9.8% 감소하였다.",
    "problems": {
        "question": "다음 사건들을 연도별로 올바르게 나열한 것은?",
        "choices": ["강력범죄 증가, 교통범죄 감소, 폭력범죄 감소", "폭력범죄 감소, 재산범죄 증가, 교통범죄 감소", "재산범죄 감소, 강력범죄 증가, 폭력범죄 증가", "교통범죄 증가, 폭력범죄 증가, 강력범죄 감소", "강력범죄 감소, 교통범죄 증가, 재산범죄 감소"],
        "answer": 1
    }
}
{
    "paragraph": "강력범죄(흉악)는 2012년 2만 8,906건에서 2021년 3만 5,126건으로 최근 10년간 21.5% 증가하였다. 반면, 교통범죄는 2012년 54만 4,201건에서 2016년 60만 970건까지 증가하였으나 그 후 큰 폭으로 감소하여 2021년에는 34만 477건으로 10년 전에 비하여 37.4% 줄었다. 폭력범죄는 2017년 이후 지속적으로 감소하는 상황으로 2012년 25만 3,115건에서 2021년 18만 460건으로 28.7%가 줄었다. 재산범죄는 2013년 66만 7,214건으로 정점을 찍은 후 감소 추세를 보이다가 2018년부터 증가 추세로 돌아섰다. 다만 2021년에는 57만 5,332건으로 10년 전 대비 9.2% 감소한 것으로 나타나 향후 추이를 지켜볼 필요가 있다. 2021년을 기준으로 전년 대비 발생 현황을 비교하여 보면, 유일하게 강력범죄(흉악)만 7.1% 증가하였고, 폭력범죄는 14.2%, 재산범죄는 12.7%, 교통범죄는 9.8% 감소하였다.",
    "problems": {
        "question": "다음 중 2012년과 2021년 사이의 범죄 통계 변화로 옳은 것은?",
        "choices": ["교통범죄는 증가하였다.", "폭력범죄는 증가하였다.", "강력범죄는 증가하였다.", "재산범죄는 증가하였다.", "모든 범죄가 증가하였다."],
        "answer": 3
    }
}
{
    "paragraph": "강력범죄(흉악)는 2012년 2만 8,906건에서 2021년 3만 5,126건으로 최근 10년간 21.5% 증가하였다. 반면, 교통범죄는 2012년 54만 4,201건에서 2016년 60만 970건까지 증가하였으나 그 후 큰 폭으로 감소하여 2021년에는 34만 477건으로 10년 전에 비하여 37.4% 줄었다. 폭력범죄는 2017년 이후 지속적으로 감소하는 상황으로 2012년 25만 3,115건에서 2021년 18만 460건으로 28.7%가 줄었다. 재산범죄는 2013년 66만 7,214건으로 정점을 찍은 후 감소 추세를 보이다가 2018년부터 증가 추세로 돌아섰다. 다만 2021년에는 57만 5,332건으로 10년 전 대비 9.2% 감소한 것으로 나타나 향후 추이를 지켜볼 필요가 있다. 2021년을 기준으로 전년 대비 발생 현황을 비교하여 보면, 유일하게 강력범죄(흉악)만 7.1% 증가하였고, 폭력범죄는 14.2%, 재산범죄는 12.7%, 교통범죄는 9.8% 감소하였다.",
    "problems": {
        "question": "2021년 강력범죄의 발생 건수는 얼마인가?",
        "choices": ["2만 8,906건", "3만 5,126건", "3만 2,898건", "18만 460건", "57만 5,332건"],
        "answer": 2
    }
}
```JSON 파싱 오류: Extra data: line 9 column 1 (char 685)
페이지 3에서 오류 발생: 'NoneType' object has no attribute 'get'

"""
# JSON 블록만 추출
json_pattern = r"\{[\s\S]*?\}\s*\}"
json_matches = re.findall(json_pattern, raw_data)


# 추출된 JSON 블록 확인
for idx, match in enumerate(json_matches):
    print(f"JSON 블록 {idx+1}:\n{match}\n")


In [ ]:
import csv
import json
import re

# 추출한 Json 블록을 복붙하여 csv 저장
# 따로 저장하면 형식이 깨져서 일단 복붙, 다른 방식의 코드가 있겠지만 빠르게 처리하기 위해 단순 복붙함
## 예시 몇개
raw_data = """
JSON 블록 1:
{
    "paragraph": "Korean Social Trends 2023안전은 크게 사회적 안전과 자연재난, 인적 사고의 세 영역으로 나누어 볼 수 있다. 사회적 안전을 위협하는 요인으로는 범죄와 테러, 자살 등이 있는데, 이 글에서는 범죄발생 현황과 범죄피해를 중심으로 살펴보았다. 자연재난은 자연현상으로 인하여 일정 규모 이상의 물적 또는 인적 손실이 발생하는 사건으로 지진, 산사태, 황사, 태풍, 호우 및 폭설, 감염병 등이 속한다. 이 글에서는 자연재해로 인한 인명 및 재산 피해를 다루었다. 인적 사고는 인간의 경제활동이나 사회활동의 결과로 시설물, 설비, 기계·기구 등이 정상적으로 작동하지 않거나, 인간이 오작동을 일으켜 발생하는 사고, 화학물질이나 에너지에 의한 인적 또는 물적 손실이 발생하는 사고 등이 속한다. 이 글에서는 사고로 인한 사망, 화재, 교통사고를 주로 살펴보았다.",
    "problems": {
        "question": "안전의 세 영역에 포함되는 것들이 아닌 것은 무엇인가?",
        "choices": ["사회적 안전", "자연재난", "인적 사고", "경제적 불안", "범죄"],
        "answer": 4
    }
}

JSON 블록 2:
{
    "paragraph": "최근 10년간 범죄발생 건수는 지속적으로 감소하고 있으며 특히 교통범죄 등 특별법범죄의 감소폭이 크다. 형법범죄는 범죄유형에 따라 차이를 보이는데, 강력범죄(흉악)의 증가 현상과 교통범죄 및 폭력범죄의 감소 현상이 두드러진다. 살인, 강도, 성폭력, 방화, 폭행ㆍ상해범죄 등 5대 강력범죄 중에서는 성폭력범죄만이 증가 추세이다.",
    "problems": {
        "question": "다음 중 최근 10년간 증가한 범죄 유형은 무엇인가?",
        "choices": ["강도", "성폭력", "살인", "방화", "폭행"],
        "answer": 2
    }
}
"""

# 패턴 정의
paragraph_pattern = r'"paragraph": "(.*?)"'
problems_pattern = r'"problems": (\{[\s\S]*?\})'

# 데이터 추출
paragraphs = re.findall(paragraph_pattern, raw_data, re.DOTALL)
problems = re.findall(problems_pattern, raw_data, re.DOTALL)

# CSV 파일 저장
output_file = "train_경제pairs.csv"
with open(output_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # CSV 헤더 작성
    writer.writerow(['paragraph', 'problems'])

    # 데이터 작성
    for para, prob in zip(paragraphs, problems):
        writer.writerow([para, prob])

print(f"데이터가 '{output_file}'에 저장되었습니다.")


In [ ]:
out = pd.read_csv("train_경제pairs.csv")
out['problems'] = out['problems'].str.replace(r'\s+', ' ', regex=True).str.replace(r'\\n', '').str.strip() # 기존 train셋과 동일하도록 전처리
out['problems'].iloc[0]

In [ ]:
out['id'] = range(1, len(out) + 1)

out['question_plus'] = np.nan


In [ ]:
out = out[['id', 'paragraph', 'problems', 'question_plus']]
out.head()


In [ ]:
# out.to_csv("train_경제pairs.csv", index=False, encoding='utf-8-sig')


In [ ]:
# df_concat = pd.concat([out, out2], ignore_index=True)
# df_concat

In [ ]:
# df_concat['id'] = range(1, len(df_concat) + 1)
# df_concat

In [ ]:
# df_concat.to_csv(".csv", index=False, encoding='utf-8-sig')


### 학습.py에 맞게 평탄화

In [ ]:
# df_concat

In [ ]:
# # 규칙 기반으로 열 분리
# df_concat['question'] = df_concat['problems'].apply(lambda x: re.search(r'"question": "(.*?)"', x).group(1))
# df_concat['choices'] = df_concat['problems'].apply(lambda x: re.search(r'"choices": (\[.*?\])', x).group(1))
# df_concat['answer'] = df_concat['problems'].apply(lambda x: re.search(r'"answer": (\d+)', x).group(1))

In [ ]:
# df_concat = df_concat.drop(columns=['problems'])
# df_concat

In [ ]:
# df_concat.columns

In [ ]:
# df_concat = df_concat[['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus']]
# df_concat.head()

In [ ]:
# df_concat.to_csv("/content/drive/MyDrive/네이버부캠/수능LLM프로젝트/train_경제와사회pairs최종.csv", index=False, encoding='utf-8-sig')


In [ ]:
# df_concat.info()

In [ ]:
# paragraph_lengths = df_concat['paragraph'].str.len()
# paragraph_lengths.describe()

In [ ]:
social = pd.read_csv("/content/drive/MyDrive/네이버부캠/수능LLM프로젝트/train_사회pairs.csv")
econo = pd.read_csv("/content/drive/MyDrive/네이버부캠/수능LLM프로젝트/train_경제pairs.csv")
print(social.head())
print(econo.head())

In [ ]:
social

In [ ]:

social['problems'] = social['problems'].str.replace(r'\s+', ' ', regex=True).str.replace(r'\\n', '').str.strip()
social['problems'].iloc[0]
social['paragraph'] = social['paragraph'].str.replace("Korean Social Trends 2023", "", regex=False)
# 기존 train셋과 동일하게 df 변경
social['id'] = range(1, len(social) + 1)

social['question_plus'] = np.nan
social = social[['id', 'paragraph', 'problems', 'question_plus']]
social.head()


In [ ]:
econo

In [ ]:
# prompt: 두 데이터프레임 합치는 코드

import pandas as pd

# 두 데이터프레임을 합칩니다.
df_concon = pd.concat([social, econo], ignore_index=True)

# 'id' 열을 재설정합니다.
df_concon['id'] = range(1, len(df_concon) + 1)
# 규칙 기반으로 열 분리
df_concon['question'] = df_concon['problems'].apply(lambda x: re.search(r'"question": "(.*?)"', x).group(1))
df_concon['choices'] = df_concon['problems'].apply(lambda x: re.search(r'"choices": (\[.*?\])', x).group(1))
df_concon['answer'] = df_concon['problems'].apply(lambda x: re.search(r'"answer": (\d+)', x).group(1))
# 결과를 확인합니다.
df_concon


In [ ]:
df_concon = df_concon.drop(columns=['problems'])
df_concon = df_concon[['id', 'paragraph', 'question', 'choices', 'answer', 'question_plus']]
df_concon

In [ ]:

# 결과를 CSV 파일로 저장합니다.
df_concon.to_csv("/content/drive/MyDrive/네이버부캠/수능LLM프로젝트/train_찐경제와사회pairs.csv", index=False, encoding='utf-8-sig')

In [ ]:
social.to_csv("/content/drive/MyDrive/네이버부캠/수능LLM프로젝트/train_사회pairs최종.csv", index=False, encoding='utf-8-sig')


### gpt 활용 퀄리티socre 부여 간단하게 필터링(학습 빠르게 하기 위함)

In [ ]:
social2 = pd.read_csv("/content/drive/MyDrive/네이버부캠/수능LLM프로젝트/Final_Social_Issues_Question_Data_with_Quality_Scores.csv")
eco2 = pd.read_csv("/content/drive/MyDrive/네이버부캠/수능LLM프로젝트/Sorted_Economics_Question_Data_with_Quality_Scores.csv")
eco2.head()

In [ ]:
# social, eco gpt 통해 score 매길 때 변형된 것이 없는지 확인
comparison = econo[['id', 'paragraph', 'problems']].merge(
    eco2[['id', 'paragraph', 'problems']],
    on=['id', 'paragraph', 'problems'],
    how='outer',
    indicator=True
)

# 출력: 동일하지 않은 데이터만 확인
mismatched_data = comparison[comparison['_merge'] != 'both']

if mismatched_data.empty:
    print("모든 데이터가 동일합니다. 변형된 데이터가 없습니다.")
else:
    print("변형된 데이터가 있습니다. 아래는 차이가 있는 행입니다:")
    print(mismatched_data)

In [ ]:
eco2['quality_score'].value_counts()

In [ ]:
print(len(eco2), len(social2))


In [ ]:
social2['quality_score'].value_counts()

In [ ]:
filtered_eco = eco2[eco2['quality_score'] > 3.5]

len(filtered_eco)

In [ ]:
filtered_so = social2[social2['quality_score'] > 4.0]

len(filtered_so)

In [ ]:
# 4점 데이터 필터링
four_point_data = filtered_eco[filtered_eco['quality_score'] == 4]

# 4점 데이터가 100개 이상인지 확인
if len(four_point_data) >= 100:
    # 4점 데이터 중 랜덤으로 100개 선택
    to_remove = four_point_data.sample(n=100, random_state=42)

    # 원본 데이터에서 선택된 100개 제거
    filtered_data = filtered_eco.drop(to_remove.index)

    print(f"4점 데이터에서 랜덤으로 100개 제거 완료. 남은 데이터 개수: {len(filtered_data)}")
else:
    print("4점 데이터가 100개 미만입니다. 제거할 수 없습니다.")

In [ ]:
combined_data = pd.concat([filtered_data, filtered_so], axis=0, ignore_index=True)
combined_data

In [ ]:
combined_data['id'] = range(1, len(combined_data) + 1)
combined_data

In [ ]:
# prompt: quality_score변수 제거 코드

# quality_score 열 제거
combined_data = combined_data.drop(columns=['quality_score','Unnamed: 0'])

# 변경된 데이터프레임 확인
combined_data.head()

In [ ]:
combined_data.to_csv("/content/drive/MyDrive/네이버부캠/수능LLM프로젝트/train_퀄리티pairs.csv", index=False, encoding='utf-8-sig')
